# ✍️ **Multi-Agent Social Media Content Generator**

## **Project Overview**

This project implements a sophisticated, multi-platform social media content generator using an **agent-based architecture** and advanced **Large Language Models** (LLMs). The goal is to produce tailored, platform-specific content (Twitter/X, Instagram, Facebook, LinkedIn, and Hashtags) from a single user input (Topic, Tone, and Language), ensuring **multilingual output** and strict adherence to platform-specific rules and professional validation requirements.


---

## 🛠️ **Key Technologies & Frameworks**

| Technology | Role in Project |
| :--- | :--- |
| **Gradio** | Provides the simple, intuitive web interface (UI) for user input and displaying generated content. |
| **LangChain** | Used to build and manage the **LLM chains**, including prompt templating, output parsing, and integrating structured data models. |
| **Groq API** | The high-speed LLM service (`llama-3.1-8b-instant`) used to perform all the generative and classification tasks. |
| **Pydantic** | Used by LangChain for **Structured Output** to reliably classify the topic's relevance for LinkedIn. |
| **Python OOP**| Implemented via **Dedicated Agent Classes** (e.g., `TwitterAgent`, `LinkedInAgent`) for clean, maintainable, and modular code design. |

---

## ⚙️ **Core Functionality and Agent Rules**

The project features a **`ContentRunner`** orchestration class that delegates tasks to specialized agents. Each agent is pre-programmed with platform-specific constraints, ensuring high-quality, formatted output.

| Platform Agent | Specific Rule Enforced |
| :--- | :--- |
| **Twitter/X Agent** | A single, highly engaging, attention-grabbing hook (under 280 characters). |
| **Instagram Agent** | A fun, two-paragraph caption, separated by a double line break. |
| **Facebook Agent** | A friendly, conversational post (2-3 sentences). |
| **LinkedIn Agent** | A professional, insightful summary (3-4 sentences). |
| **Hashtag Agent** | Exactly 3 relevant, highly specific hashtags. |

### 💼 **Critical Validation Rule (LinkedIn)**

The runner includes a crucial Pydantic-driven **Topic Classification** step to enforce professional standards:

* The LinkedIn post is generated **only** if the input topic is classified as **business, tech, finance, or career relevant**.
* If the topic is *not* relevant (e.g., purely recreational), the LinkedIn output field is returned as an **empty string** (`""`).

---

## 🔑 **Setup and Prerequisites**

To run this notebook, you **MUST** obtain and set your `GROQ_API_KEY`. This key is essential for the `ChatGroq` class to communicate with the Large Language Model service.

1.  **Install Libraries:** Uncomment and run the `!pip install` command block.
2.  **Set API Key:** Replace the `YOUR_GROQ_API_KEY_HERE` placeholder in the code or set the `GROQ_API_KEY` as a Colab secret/environment variable.

# **Initial Setup**

In [ ]:
#
# Installation (Uncomment and run if libraries are missing in your environment)
#

#
# The option -q of pip give less output.
#
# The Option is additive. In other words, you can use it up to 3 times (corresponding to WARNING, ERROR, and CRITICAL logging levels).
#
# So:
#   -q   means display only the messages with WARNING,ERROR,CRITICAL log levels
#   -qq  means display only the messages with ERROR,CRITICAL log levels
#   -qqq means display only the messages with CRITICAL log level
#
# !pip install -qqq gradio langchain-groq langchain_core pydantic

In [ ]:
# #
# # Import Libraries
# #
# import os                  # Used to retrieve the GROQ_API_KEY from the system environment variables.
# import gradio as gr        # THE UI LIBRARY: Used to create the web interface (Inputs, Buttons, Outputs) quickly.
# from typing import Literal # Used for type hinting, specifically to limit a variable's value to a specific set of strings.

# #
# # NOTE
# # While these are imported, they are not strictly used when calling a remote API like Groq.
# #
# import transformers # Core library for working with models (often used for tokenization/local models).
# import torch        # The leading deep learning framework, often used for model computation.
# import accelerate   # A library to easily run large model training/inference across different hardware (GPUs).

# #
# # LangChain Imports - Core libraries for building complex LLM applications
# #
# from langchain_groq import ChatGroq                       # The specific connector (wrapper) class for the Groq API.
# from langchain_core.prompts import ChatPromptTemplate     # The foundation for creating structured, multi-role prompts.
# from langchain_core.output_parsers import StrOutputParser # A parser to simply return the LLM's response as a basic string.

# # from langchain_core.pydantic_v1 import BaseModel, Field   # Pydantic classes for reliable, structured data handling.
# # WARNING FIX: Updated import to eliminate the LangChainDeprecationWarning.
# # We now import Pydantic components directly from the Pydantic library.

# from pydantic import BaseModel, Field                     # Pydantic classes for reliable, structured data handling.

In [ ]:
#
# Import Common Libraries
#
import os         # Used to retrieve the GROQ_API_KEY from the system environment variables.
import sys        # Used for environment detenction and exiting the script
import subprocess # For dependency installantion to work from command line (!pip raises syntax error at compile time)

#
# Conditional Dependency Installation for Notebook and Script Compatibility
#
try:
  import gradio as gr        # THE UI LIBRARY : Used to create the web interface (Inputs, Buttons, Outputs) quickly.
  from typing import Literal # Used for type hinting, specifically to limit a variable's value to a specific set of strings.

  #
  # NOTE
  # While these are imported, they are not strictly used when calling a remote API like Groq.
  #
  import transformers # Core library for working with models (often used for tokenization/local models).
  import torch        # The leading deep learning framework, often used for model computation.
  import accelerate   # A library to easily run large model training/inference across different hardware (GPUs).

  #
  # LangChain Imports - Core libraries for building complex LLM applications
  #
  from langchain_groq import ChatGroq                       # The specific connector (wrapper) class for the Groq API.
  from langchain_core.prompts import ChatPromptTemplate     # The foundation for creating structured, multi-role prompts.
  from langchain_core.output_parsers import StrOutputParser # A parser to simply return the LLM's response as a basic string.

  # from langchain_core.pydantic_v1 import BaseModel, Field   # Pydantic classes for reliable, structured data handling.
  # WARNING FIX: Updated import to eliminate the LangChainDeprecationWarning.
  # We now import Pydantic components directly from the Pydantic library.

  from pydantic import BaseModel, Field                     # Pydantic classes for reliable, structured data handling.
except ImportError:
    #
    # This block executes ONLY if the initial imports fail
    # (i.e., dependencies are missing)
    #
    print("Dependencies not found. Attempting to install...")

    #
    # Check if running in an interactive notebook environment (Colab/Jupyter)
    #
    if '__package__' in globals() or 'ipykernel' in sys.modules:
      print("Notebook environment detected. Using subprocess.check_call...")

      #
      # Use notebook-specific command for quiet installation
      #
      # !pip install -qqq gradio langchain-groq langchain_core pydantic
      #
      # Use subprocess to run pip, which works in both notebook and
      # script environments
      #
      subprocess.check_call(
                             [
                               sys.executable,
                               "-m",
                               "pip",
                               "install",
                               "-qqq",
                               "gradio",
                               "langchain-groq",
                               "langchain_core",
                               "pydantic",
                               "transformers",
                               "torch",
                               "accelerate"
                             ]
                           )
    else:
      print("")
      print("**Dependencies are missing.**")
      print("Please install required libraries manually using:")
      print("pip install gradio langchain-groq langchain_core pydantic transformers torch accelerate")

      #
      # Exit if dependencies are missing in a script environment
      #
      sys.exit(1)

    # Re-import after successful installation (necessary for notebooks)
    try:
      import os
      import sys
      import gradio as gr
      from typing import Literal

      #
      # NOTE
      # While these are imported, they are not strictly used when calling a remote API like Groq.
      #
      import transformers
      import torch
      import accelerate

      from langchain_groq import ChatGroq
      from langchain_core.prompts import ChatPromptTemplate
      from langchain_core.output_parsers import StrOutputParser

      from pydantic import BaseModel, Field
    except Exception as e:
      print(f"Failed to load dependencies after installation: {e}")
      sys.exit(1)

In [ ]:
#
# Wrapper to run the code in an non-google colab environment
# This block ensures the code can access secrets/environment variables
# whether running in Google Colab or a standard Python environment
#
try:
  from google.colab import userdata
  #
  # If running in Google Colab, the 'userdata' object is successfully
  # imported.
  # It is used to securely fetch secrets from the Colab Secrets Manager
  #
except ImportError:
  #
  # Dummy class for non-Colab environments
  #
  # If an ImportError occurs (meaning we are NOT in Colab), this 'except' block
  # executes to define a fallback mechanism
  #
  class DummyUserdata:
    #
    # The 'get' method is implemented to mimic the Colab 'userdata.get()'
    # functionality, allowing the rest of the code to remain the same
    #
    def get(self, name):
      #
      # Instead of using the Colab Secrets Manager, this method looks for the
      # variable (name) in the operating system's standard environment
      # variables using 'os.getenv()'
      #
      return os.getenv(name)

  #
  # An instance of the dummy class is created and assigned to 'userdata'
  # This makes 'userdata.get(name)' work correctly in both environments
  #
  userdata = DummyUserdata()

# **Configuration**

In [ ]:
SECRET_NAME     = "GROQ_API_KEY"
UNKNOWN_API_KEY = "UNKNOWN_API_KEY"
GROQ_MODEL      = "llama-3.1-8b-instant" # "llama3-8b-8192" # OLD, DECOMMISSIONED MODEL (Causing the Error)

SHARE           = True
NO_SHARE        = False

In [ ]:
#
# Fetch API Key Value
#
# Make it more generic by setting an OS Environment Variable
# after fetching from Google Colab
#

#
# Retrieve the API key securely from Colab Secrets.
#
try:
  API_KEY = userdata.get(SECRET_NAME)

  #
  # Set it as an environment variable so the rest of your script
  # can access it using os.getenv()
  #
  if API_KEY:
    os.environ[SECRET_NAME] = API_KEY
    print("✅ API Key successfully retrieved from Colab Secrets and set as environment variable.")
  else:
    print(f"❌ Error: Secret '{SECRET_NAME}' was found but is empty. Please check the value.")
except Exception as e:
  print(f"❌ Error retrieving Secret '{SECRET_NAME}'. Ensure it is saved correctly in the Colab sidebar.")
  print(f"Detailed error: {e}")

#
# Rest of the program, can now use
#
# This also ensures that the program works seemlessly in a non-google colab
# environment
#
API_KEY = os.getenv(SECRET_NAME, UNKNOWN_API_KEY)
print(f"Key check (last 3 chars): ...{API_KEY[-3:] if API_KEY and len(API_KEY) > 3 else 'N/A'}")

# **Relevance Model (Pydantic Schema)**

In [ ]:
#
# LinkedIn Relevance Pydantic Model and Classifier
#
class TopicClassification(BaseModel):
  #
  # CLASS: Pydantic Model
  #   Pydantic is a Python library that uses Python type hints for data
  #   validation, parsing, and settings management, making code more robust
  #   and reliable.
  #
  #   The Ellipsis (...), one use below, means Mandatory
  #
  #   WHAT IT DOES  : Defines the EXACT structure of the JSON output we want
  #                   from the LLM.
  #   WHY WE USE IT : LangChain uses this schema to instruct the LLM to return
  #                   a clean, reliable JSON object, which is crucial for
  #                   decision-making (the LinkedIn rule)
  #
  isTechBizRelevant : Literal["Yes", "No"] = Field(
                                                    #
                                                    # The field name the LLM must use in its JSON output.
                                                    # Detail on '...' (Ellipsis):
                                                    # In Pydantic, the Ellipsis '...' is often used as a special placeholder
                                                    # to indicate that a field **MUST** be provided (it is REQUIRED) and has no default value.
                                                    ...,
                                                    # Type Hint: Forces the field value to be either the string "Yes" or the string "No".
                                                    #

                                                    #
                                                    # This description is passed directly to the LLM to guide its classification output.
                                                    #
                                                    description = "Must be 'Yes' if the topic is primarily about technology, business, finance, career development, or professional skills. Otherwise, respond 'No'."
                                                  )

In [ ]:
def classifyTopic(llm: ChatGroq, topic: str) -> str:
  #
  # Function : classifyTopic
  #  Parameters
  #  llm: ChatGroq -> Parameter llm of type ChatGroq
  #  topic:str     -> Parameter topic of type str
  #
  #  -> str        -> Return a strin
  #
  #  WHAT IT DOES  : Determines if the user's topic is appropriate for a
  #                  professional LinkedIn post.
  #  WHY WE USE IT : This function enforces the validation rule:
  #                  'LinkedIn post should be an empty string if the topic is not business/tech relevant'
  #
  #  RETURNS       : A string: either "Yes" or "No".
  #

  # LangChain Component Detail: ChatPromptTemplate.from_messages
  #   WHAT IT DOES  : Creates a reusable, structured blueprint for the conversation.
  #   WHY WE USE IT : LLMs work best with role-based input.
  #                   This allows us to define:
  #                     1. A 'system' role
  #                        (telling the model *how* to behave: "You are a specialized classifier").
  #                     2. A 'user' role
  #                        (passing the variable input: the topic to classify).
  #
  # WHAT IT RETURNS  : A PromptTemplate object.
  #

  #
  # CRITICAL FIX: Made the system prompt more robust by providing examples
  #
  classifierPrompt = ChatPromptTemplate.from_messages(
                                                       [
                                                         ("system", "You are a specialized classifier. Determine if the topic is relevant to a professional audience. This includes **business strategy, human resources (HR), finance, technology, AI, career development, professional skills, and organizational policy**. Specifically, topics about **employee retention, productivity, work week structure (e.g., 4-day work week), remote work, leadership, and company policy** MUST be classified as 'Yes'. Respond ONLY with the strict JSON format defined."),
                                                         ("user", f"Classify the following topic: '{{topic}}'") # {topic} is a variable filled later.
                                                       ]
                                                     )

  # classifierPrompt = ChatPromptTemplate.from_messages(
  #                                                      [
  #                                                        ("system", "You are a specialized classifier. Analyze the user's topic and determine if it is relevant to a professional audience (e.g., **business strategy, human resources, finance, career development, or professional skills**). Topics about **employee retention, productivity, or work policy** must be classified as 'Yes'. Respond only with the strict JSON format defined."),
  #                                                        ("user", f"Classify the following topic: '{{topic}}'") # {topic} is a variable filled later.
  #                                                      ]
  #                                                    )

  # classifierPrompt = ChatPromptTemplate.from_messages(
  #                                                      [
  #                                                        ("system", "You are a specialized classifier. Analyze the user's topic and determine if it is relevant to a professional audience (business, technology, finance, career, professional skills). Respond only with the strict JSON format defined."),
  #                                                        ("user", f"Classify the following topic: '{{topic}}'") # {topic} is a variable filled later.
  #                                                      ]
  #                                                    )

  #
  # Chain Construction: Prompt | LLM (with forced JSON output)
  #
  # llm.with_structured_output(TopicClassification): This is the magic! It forces the LLM to format its response
  # STRICTLY according to the TopicClassification Pydantic model.
  #
  # --------------------------------------------------------
  # LangChain Expression Language (LCEL) Chain Construction
  # --------------------------------------------------------
  #
  # LCEL allows us to define a data flow pipeline using the pipe operator (|).
  #
  # The output of the left side is automatically fed as the input to the right side.
  #
  #   1. Left Side     : classifierPrompt (Output : Formatted Prompt)
  #   2. Pipe Operator : | (Data Flow Sequencer)
  #   3. Right Side    : llm.with_structured_output(TopicClassification)
  #
  # Detail on the Right Side (LLM Executor):
  #   llm                                          : This is the instantiated ChatGroq model.
  #   .with_structured_output(TopicClassification) : This method is the magic!
  #
  #   WHAT IT DOES  : It modifies the LLM's behavior by telling it to
  #                   **force** its output into a JSON structure that matches
  #                   the Pydantic model TopicClassification.
  #   WHY WE USE IT : Ensures the output is clean, reliable JSON, preventing
  #                   parsing errors.
  #
  classificationChain = classifierPrompt | llm.with_structured_output(TopicClassification)

  try:
    #
    # Invoke the chain, passing the topic variable
    #
    classification = classificationChain.invoke({"topic" : topic})

    #
    # Access the structured Pydantic object's field and return its value
    # ("Yes" or "No")
    #
    return classification.isTechBizRelevant
  except Exception as e:
    #
    # Fallback for API or parsing errors, defaulting to safe/non-relevant to
    # skip LinkedIn post
    #
    print(f"Classification failed (Defaulting to 'No'): {e}")
    return "No"

# **Social Media Agents**

## **Base Class and Implementation**

**Method Conventions**


---


**The Underscore Convention**  
When a method or attribute name is prefixed with a **single underscore (`_`)**, it signals to other developers that the method is intended for **internal use** by the class and should not be directly called from outside the class.

**In Python, methods prefixed with `_` are called "protected" members.** They are still technically accessible (you can call them, e.g., `runner._initializeLLM()`), but doing so violates the developer's intent and can lead to bugs if the class's internal logic changes later.

**No Underscore Convention**  
Methods without an underscore prefix are considered **public** members. They define the official **interface** or public contract of the class. These methods are safe and intended to be called directly by other parts of the program.

In summary, the underscore is a way developers communicate **intent** about how their classes should be used: **Public** methods define what the class does, and **Private/Protected (`_`)** methods define how the class does it internally.

In [ ]:
#
# Base class defining the common structure for all social media agents
#
# WHAT IT DOES  : A template class that provides common setup (LLM, prompt) for
#                 all agents
# WHY WE USE IT : Reduces code duplication by having all agents inherit the
#                 standard chain creation and generation logic.
#                 This is the Object-Oriented Programming (OOP)
#                 principle of Inheritance.
#
class baseAgent:
  #
  # Constructor method: Sets up the Agent's properties
  #
  def __init__(self, llm : ChatGroq, platform : str, rules : str):
    self.platform = platform               # e.g., "Twitter/X"
    self.rules    = rules                  # The specific rules (e.g., "under 280 characters")
    self.chain    = self._createChain(llm) # Initializes the LangChain pipeline immediately

  def _createChain(self, llm : ChatGroq):
    #
    # METHOD        : _create_chain (A factory method)
    # WHAT IT DOES  : Builds the specific LangChain pipeline for this platform.
    # WHY WE USE IT : Encapsulates the prompt definition logic.
    # RETURNS       : A LangChain Runnable sequence (Prompt | LLM | Parser).
    #
    PromptTemplate = ChatPromptTemplate.from_messages(
                                                       [
                                                         (
                                                           "system",
                                                           f"You are an expert social media content writer for {self.platform}."
                                                           f"Your post must **STRICTLY** follow these rules: {self.rules}"
                                                           f"The final output must only contain the generated content."
                                                         ),
                                                         (
                                                           "user",
                                                           "Generate content for the following topic: {topic}. Target tone: {tone}. **Output MUST be in {language}.**"
                                                         ) # CHANGE: Added {language} instruction
                                                       ]
                                                     )

    #
    # LangChain Component Detail: | (Pipe Operator)
    #
    # WHAT IT DOES    : Chains together LangChain Runnable objects (Sequencing).
    # WHY WE USE IT   : Creates a simple, readable pipeline:
    #                   Prompt -> LLM -> OutputParser.
    # StrOutputParser : Simply converts the LLM's text output into a standard
    #                   Python string.
    #
    return PromptTemplate | llm | StrOutputParser()

  #
  # CHANGE : Added 'language' parameter to the public generate method
  #
  def generate(self, topic : str, tone : str, language : str) -> str:
    #
    #  METHOD        : generate
    #  WHAT IT DOES  : Executes the LangChain pipeline using the provided
    #                  topic and tone.
    #  WHY WE USE IT : Provides a clean, standardized public interface for
    #                  the runner to call.
    #  RETURNS       : The generated text content as a string.
    #

    #
    # Invoke the chain, substituting the {topic} and {tone} variables
    # in the prompt
    #
    return self.chain.invoke(
                              {
                                "topic"    : topic,
                                "tone"     : tone,
                                "language" : language # NEW: Pass language to the prompt
                              }
                            )

**Define Agents for each platform inheriting from BaseAgent. They only need to set their unique rules**

## **Twitter Agent**

In [ ]:
class twitterAgent(baseAgent):
  #
  # Agent for generating Twitter/X content with its specific rule
  #
  def __init__(self, llm : ChatGroq):
    rules = "A single, highly engaging, attention-grabbing Twitter/X Hook (under 280 characters)."
    super().__init__(llm, "Twitter/X", rules) # Calls the parent (baseAgent) constructor.

## **Instagram Agent**

In [ ]:
class instagramAgent(baseAgent):
  #
  # Agent for generating Instagram content with the two-paragraph rule
  #
  def __init__(self, llm : ChatGroq):
    rules = "A fun, two-paragraph caption for Instagram. Separate the paragraphs with a double line break."
    super().__init__(llm, "Instagram", rules)

## **Facebook Agent**

In [ ]:
class facebookAgent(baseAgent):
  #
  # Agent for generating Facebook content with the friendly, conversational rule
  #
  def __init__(self, llm : ChatGroq):
    rules = "A friendly, conversational post for Facebook (2-3 sentences)."
    super().__init__(llm, "Facebook", rules)

## **LinkedIn Agent**

In [ ]:
class linkedInAgent(baseAgent):
  #
  # Agent for generating LinkedIn content (3-4 sentences)
  #
  def __init__(self, llm : ChatGroq):
    #
    # The conditional "empty string" logic is handled by the ContentRunner, not here.
    #
    # rules = "A professional, insightful summary suitable for a LinkedIn post (3-4 sentences)."
    rules = (
              "Generate a professional, insightful summary suitable for a LinkedIn post (3-4 sentences).\n"
              "**Tone & Persona:** Act as a LinkedIn Content Strategist. Your voice must be **authoritative, clear, and empathetic**.\n"
              "Maintain a professional yet approachable attitude.\n"
              "**Style Constraints:** Use contractions, and strictly limit exclamation points to a maximum of one per post.\n"
              "**Vocabulary Constraints:** Avoid financial jargon, and never use the words 'disruptive' or 'guru'.\n"
            )
    # rules = (
    #           "You are a professional social media copywriter. Generate compelling content for the user's topic for Twitter/X, Instagram, Facebook, and LinkedIn.\n"
    #           "**LinkedIn Content Rule:** Before generating, assess if the user's topic is suitable for a professional/business audience (e.g., IT, finance, careers, company culture, technology).\n"
    #           "- If the topic IS suitable (e.g., 'Influence of AI in IT'), generate the professional summary.\n"
    #           "- If the topic is NOT suitable (e.g., 'Effects of Honey + Hot Water'), provide an **empty string ("")** for the 'linkedin_article_summary' field."
    #         )

    super().__init__(llm, "LinkedIn", rules)

## **HashTag Agent**

In [ ]:
# class hashTagAgent(baseAgent):
#   #
#   # Agent for generating exactly three highly specific hashtags
#   #
#   def __init__(self, llm : ChatGroq):
#     rules = "Exactly 3 relevant, highly specific hashtags, separated by spaces (e.g., #productivitytips #remotework #ai)"
#     super().__init__(llm, "Hashtag Generator", rules)

#
# CRITICAL FIX: Hashtag Agent with Overridden Chain
#
class hashTagAgent(baseAgent):
  #
  # Agent for generating exactly three highly specific hashtags
  #
  def __init__(self, llm : ChatGroq):
    rules = "Exactly 3 relevant, highly specific hashtags, separated by spaces (e.g., #productivitytips #remotework #ai)"
    super().__init__(llm, "Hashtag Generator", rules)

  #
  # NEW METHOD: Override baseAgent's chain creation
  #
  def _createChain(self, llm : ChatGroq):
    #
    # This new system prompt explicitly makes the agent a "generation engine"
    # and uses all-caps instructions to ensure compliance.
    #
    promptTemplate = ChatPromptTemplate.from_messages(
                                                       [
                                                         (
                                                           "system",
                                                           f"You are a specialized hashtag generation engine. Your sole task is to output a string consisting of **EXACTLY 3** relevant, highly specific hashtags, separated by spaces. Your output MUST NOT contain any other characters, descriptions, or introductory phrases. The rule is: {self.rules}"
                                                         ),
                                                         (
                                                           "user",
                                                           "Generate hashtags for the topic: {topic}. Target tone: {tone}. **Output MUST be in {language}.**" # CHANGE: Added {language} instruction
                                                         )
                                                       ]
                                                     )
    return promptTemplate | llm | StrOutputParser()

## **Content Runner Agent**
**Content Runner (Orchestration)**

In [ ]:
class contentRunner:
  #
  # CLASS: ContentRunner
  # WHAT IT DOES  : Acts as the central controller (the 'Runner').
  #                 It manages the LLM instance, initializes all agents, and
  #                 orchestrates the entire content generation workflow.
  # WHY WE USE IT : Separates the setup/logic from the Gradio UI and centralizes
  #                 the crucial LinkedIn conditional check.
  #
  def __init__(self, groqAPIKey : str):
    #
    # Constructor: Dtores the key and immediately sets up the LLM and Agents.
    #
    self.groqAPIKey = groqAPIKey
    self.llm        = self._initializeLLM()
    self.agents     = self._initializeAgents()

  def _initializeLLM(self):
    #
    # Initializes the Groq LLM instance, validating the key first
    # Raise error, if the API Key is UNKNOWN
    #
    if self.groqAPIKey == UNKNOWN_API_KEY:
      #
      # Use a ValueError to signal a critical, unrecoverable error back to
      # the caller
      #
      raise ValueError("GROQ_API_KEY is missing. Please make sure it's set")

    #
    # Initializes the LangChain ChatGroq wrapper
    #
    return ChatGroq(
                     temperature  = 0.7,             # Controls randomness (0.0 = deterministic, 1.0 = creative)
                     groq_api_key = self.groqAPIKey,
                     model        = GROQ_MODEL       # Specifies the fast, chosen model
                   )

  def _initializeAgents(self):
    #
    # Instantiates all platform-specific agents using the initialized LLM
    #
    return {
             "Twitter"   : twitterAgent(self.llm),
             "Instagram" : instagramAgent(self.llm),
             "Facebook"  : facebookAgent(self.llm),
             "LinkedIn"  : linkedInAgent(self.llm),
             "HashTags"  : hashTagAgent(self.llm)
           }

  #
  # CHANGE: Added 'language' parameter
  #
  def runAll(self, topic : str, tone : str, language : str):
    #
    # METHOD        : runAll
    # WHAT IT DOES  : The main execution method. Runs classification, then all
    #                 agents sequentially
    # WHY WE USE IT : This is the single entry point to generate all content
    # RETURNS       : A tuple containing a dictionary of all results and a
    #                 status message
    #

    #
    # Critical Classification check
    # (Determines if the LinkedIn Agent should run)
    #
    isRelevant = classifyTopic(self.llm, topic)
    relevanceCheck = f"Topic Relevance Check: **{isRelevant}**"

    #
    # Sequential content generation using the agent instances
    #
    agentResults = {}

    #
    # Run agents for platforms without conditional rules
    # CHANGE: Passed 'language' to all agent generation calls
    #
    agentResults["Twitter"]   = self.agents["Twitter"].generate(topic, tone, language)
    agentResults["Instagram"] = self.agents["Instagram"].generate(topic, tone, language)
    agentResults["Facebook"]  = self.agents["Facebook"].generate(topic, tone, language)
    agentResults["HashTags"]  = self.agents["HashTags"].generate(topic, tone, language)

    #
    # Apply LinkedIn Conditional Logic (The primary rule enforcement)
    #
    if isRelevant == "Yes":
      #
      # CHANGE: Passed 'language' to LinkedIn generation call
      #
      agentResults["LinkedIn"] = self.agents["LinkedIn"].generate(topic, tone, language)
    else:
      #
      # If not relevant, fulfill the rule by setting the output to an
      # empty string.
      #
      agentResults["LinkedIn"] = "Content Skipped : This topic was automatically deemed **unsuitable for a professional LinkedIn audience**, so content generation was skipped for this platform."
      relevanceCheck += "\n*(LinkedIn Post skipped due to non-business/tech relevance rule.)*"

    return agentResults, relevanceCheck

## **Generate Content**

**Gradio Runner Function and Setup**

In [ ]:
#
# CHANGE: Added 'language' parameter
#
def generateContent(topic : str, tone : str, language : str):
  #
  # FUNCTION      : generateContent
  # WHAT IT DOES  : This function acts as the 'glue' between the Gradio UI and
  #                 the ContentRunner logic
  # WHY WE USE IT : Gradio requires a Python function to connect its button clicks
  #                 to the back-end code
  # RETURNS       : A tuple of 6 strings, which Gradio maps to the 6 output
  #                 components
  #
  try:
    #
    # Initialize the Runner
    # This is where the LLM is initialized and the API Key is checked
    #
    runner = contentRunner(API_KEY)

    #
    # Execute the main orchestration logic
    # CHANGE: Passed 'language' to runAll
    #
    runResults, statusMessage = runner.runAll(topic, tone, language)

    #
    # Return results in the exact order Gradio expects
    # (as defined in the .click() call)
    #
    return (
             runResults["Twitter"],
             runResults["Instagram"],
             runResults["Facebook"],
             runResults["LinkedIn"],
             runResults["HashTags"],
             statusMessage
           )
  except ValueError as e:
    #
    # Handles the specific error raised if API_KEY is UNKNOWN
    #
    return ("","","","","", f"⚠️ {str(e).replace(API_KEY, '')}")
  except Exception as e:
    #
    # Handles general errors (e.g., network issues, Groq API call failure)
    #
    return ("","","","","", f"❌ Error communicating with LLM: {str(e).replace(API_KEY, '')}")

## **Interface**

**Gradio Interface**


---

**The Purpose of the `with` Statement**  
The primary goal of the `with` statement is to simplify the `try...finally` pattern, making code cleaner and safer.

* **Setup (`__enter__`)**  
When Python executes the with statement, it calls a special method on the object called `__enter__`. This method performs the necessary setup (e.g., opening a file, starting a database transaction, or initiating a UI block). The return value of this method is assigned to the variable after the `as` keyword (e.g., `as demo` or `as file`).

* **Teardown (`__exit__`)**  
Once the code block finishes—whether normally or because an exception was raised—Python guarantees that it will call another special method, `__exit__`. This method handles the cleanup (e.g., closing the file, committing the transaction, or finalizing the UI block).

This guarantee prevents resource leaks (like an open file handle) that could lead to system instability or errors.

---


**Example : Gradio Blocks (Structural Context)**

The `with` construct in the Social Media Generator is used to define a **hierarchical structure** and **scoping** for the web interface, ensuring that UI components are correctly grouped inside their parent containers (like rows and columns).

| Method | Purpose |
| :--- | :--- |
| `__enter__` | Sets the object as the currently active container. |
| `__exit__` | Finalizes the container and restores the previous parent container. |


In [ ]:
with gr.Blocks(
                theme = gr.themes.Soft(),
                title = "(4-in-1) - Social Media Content Generator"
              ) as contentGenerator:
  #
  # gr.Blocks: The container for the whole User Interface
  #
  gr.Markdown("## ✍️ Multi-Agent (4-in-1) Multi-Lingual Social Media Content Generator")
  # gr.Markdown("### Using (LangChain + Groq)")

  with gr.Row(): # Groups components horizontally
    with gr.Column(scale = 1): # Groups components vertically within the row
      topicInput = gr.Textbox(
                               label = "Content Topic",
                               value = "The benefits of switching to a 4-day work week for employee retention." # "Why remote work is changing company culture."
                             )

      toneInput = gr.Textbox(
                              label = "Target Style/Tone",
                              value = "Uplifting and professional."
                            )

      #
      # NEW COMPONENT: Language Dropdown
      #
      languageInput = gr.Dropdown(
                                   label   = "Output Language",
                                   choices = ["English", "Spanish", "French", "German", "Japanese", "Mandarin", "Portuguese", "Tamil"], # Example supported languages
                                   value   = "English" # Set a default
                                )

      generateButton = gr.Button("Generate Content", variant = "primary")

      with gr.Column(scale = 2):
        statusOutput = gr.Markdown("Status : Ready. Please enter a topic, tone and language.")

  gr.Markdown("---")
  gr.Markdown("## 📝 Generated Content")

  #
  # Outputs are grouped by format type
  #
  # -----------------------------------------------------------
  # Layout Overview: Grouping for Visual Aesthetics & Efficiency
  # -----------------------------------------------------------
  #
  # Top Row: Inputs and Status
  # --------------------------
  # - The first gr.Row() groups:
  #     * gr.Column(scale=1): Inputs and action button
  #     * gr.Column(scale=2): Status / output display
  # - This cleanly separates the control panel from the results area.
  #
  # Second Row: Hashtags and LinkedIn
  # ---------------------------------
  # - Hashtags: short, single-line text
  # - LinkedIn Post: medium-length (approx. 4 lines)
  # - Placing them side-by-side uses horizontal space efficiently.
  #
  # Third Row: Twitter and Facebook
  # -------------------------------
  # - Twitter/X: short (≈2 lines)
  # - Facebook: medium-short (≈3 lines)
  # - Grouping these quick-read outputs together improves scanability.
  #
  # Instagram (Standalone)
  # ----------------------
  # - insta_out is placed outside any gr.Row().
  # - It's the longest output (≈5 lines) and benefits from full-width display.
  # - This makes longer, multi-paragraph content easier to read.
  #
  # Instagram output (full-width)
  #
  # This grouping maximizes space efficiency and presents related content types
  # together, making the UI easier to scan
  #
  with gr.Row():
    hashTagOutput  = gr.Textbox(
                                 label       = "Hashtags (Exactly 3 highly specific)",
                                 interactive = False
                                 # lines       = 2
                               )
    linkedInOutput = gr.Textbox(
                                 label       = "💼 LinkedIn Post (Professional, 3-4 sentences - **Skipped if not business/tech**)",
                                 interactive = False,
                                 lines       = 4
                               )

  with gr.Row():
    twitterOutput  = gr.Textbox(
                                 label       = "🐦Twitter/X Hook (Single, under 280 chars)",
                                 interactive = False,
                                 lines       = 2
                               )
    facebookOutput = gr.Textbox(
                                 label       = "👍 Facebook Post (Friendly, 2-3 sentences)",
                                 interactive = False,
                                 lines       = 3
                               )

  instagramOutput = gr.Textbox(
                                label       = "📸 Instagram Caption (Fun, Two Paragraphs, Double Line Break)",
                                interactive = False,
                                lines       = 5
                              )

  #
  # Connect the button click event to the Python function
  #
  generateButton.click(
                        fn      = generateContent, # The function to run when the button is clicked
                        inputs  = [
                                    topicInput,
                                    toneInput,
                                    languageInput # NEW: Added language input
                                  ], # List of inputs to pass to generate_content
                        outputs = [
                                    twitterOutput,
                                    instagramOutput,
                                    facebookOutput,
                                    linkedInOutput,
                                    hashTagOutput,
                                    statusOutput
                                  ] # List of outputs to update
                      )

## **Launch**

**Possible Tones that we can use**

| Tone Adjectives | Example Prompt Instruction |
| :--- | :--- |
| **Professional** and **Authoritative** | "Write an analysis of the latest industry report. Use an authoritative and expert tone." |
| **Friendly** and **Approachable** | "Draft a post about my weekend project. Keep the tone casual and encouraging." |
| **Inspiring** and **Visionary** | "Generate a post on the future of work. The tone should be inspirational and forward-looking." |
| **Candid** and **Humble** | "Write about a recent business failure. Maintain a candid and humble tone, focusing on lessons learned." |

In [ ]:
#
# Launch the Content Generator
#
contentGenerator.launch(
                         quiet = True,
                         share = False # SHARE / NO_SHARE - Public Sharable Link
                       )